In [29]:
# 필요한 라이브러리 설치 및 임포트
!pip install pandas
!pip install lxml
import pandas as pd
import os
import glob
import re

# 컬럼 안의 문자열 최대 표시 길이 설정 (None이면 제한 없음)
pd.set_option('display.max_colwidth', None)
# 출력할 컬럼 개수 제한 해제
pd.set_option('display.max_columns', None)
# 출력할 행 개수 제한 해제
pd.set_option('display.max_rows', None)


## 1. 학기별 개설교과목정보 데이터 합치기

In [30]:
files = glob.glob("data/*.xls")

all_data = []

for file in files:
    # HTML 테이블 읽기
    df = pd.read_html(file)[0]
    
    # 첫 번째 행을 컬럼명으로 지정
    df.columns = df.iloc[0]   # 0번 행을 컬럼명으로
    df = df.drop(index=0)     # 0번 행 삭제
    
    # 학기 정보 컬럼 추가
    semester = os.path.basename(file).replace("-sis.xls", "")
    df["개설학기정보"] = semester
    
    all_data.append(df)

# 모든 파일 병합
df_all = pd.concat(all_data, ignore_index=True)

df_all.head(3)


,학년도,학기,소속,학과,과목번호,분반,과목명,학점,수업시간/강의실,시간,교수진,수강생수,영어강의,중국어강의,승인과목,CU과목,홀짝구분,국제학생,Honors과목,공학인증,시험일자,수강대상,권장학년,수강신청 참조사항,과목 설명,비고,NaN,개설학기정보
0,2024 학년도,2학기,대학,게페르트국제학부,AAS1001,01,아시아학 개론,3.0,"화,목 12:00~13:15 [J107]",3.0,김소연,0000000048,O,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,전학년,1-4학년,NaN,NaN,NaN,2024-2
1,2024 학년도,2학기,대학,아트&테크놀로지학과,AAT2002,01,Humanities & creativity,3.0,월 13:30~16:15 [X514],3.0,정지수,0000000021,O,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,전학년,1-4학년,NaN,구) ANT2003,NaN,2024-2
2,2024 학년도,2학기,대학,아트&테크놀로지학과,AAT2003,01,Intro to Digital Arts,3.0,월 13:30~16:15 [X513],3.0,이형기,0000000031,O,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"2,3,4학년",1-4학년,NaN,구) ANT2004,NaN,2024-2


## 2. 필요한 열 선택

In [31]:
def make_major_df(df_all, codes=None, prefix=None):
    if codes is not None:
        df_major = df_all[df_all["과목번호"].isin(codes)]
    elif prefix is not None:
        df_major = df_all[df_all["과목번호"].str.startswith(prefix)]
    else:
        raise ValueError("codes 또는 prefix 중 하나를 지정해야 합니다.")

    result = []
    for code in df_major["과목번호"].unique():
        subset = df_major[df_major["과목번호"] == code]
        sems = ", ".join(sorted(subset["개설학기정보"].unique()))
        names = ", ".join(sorted(subset["과목명"].unique()))
        profs = ", ".join(sorted(subset["교수진"].dropna().unique()))
        result.append({
            "과목번호": code,
            "과목명": names,
            "교수": profs,
            "개설학기": sems
        })
    return pd.DataFrame(result)


## 3. 전공 데이터프레임 계절별로 만들기

In [32]:
def make_major_dfs(df_all, major_name, codes=None, prefix=None):
    import builtins

    # 정리된 전공 데이터프레임 생성
    df_major = make_major_df(df_all, codes=codes, prefix=prefix)
    builtins.__dict__[f"df_{major_name}"] = df_major

    # 계절별 데이터 생성
    builtins.__dict__[f"df_{major_name}_spring"] = df_major[df_major["개설학기"].str.contains("-1", na=False)]
    builtins.__dict__[f"df_{major_name}_fall"] = df_major[df_major["개설학기"].str.contains("-2", na=False)]
    builtins.__dict__[f"df_{major_name}_summer"] = df_major[df_major["개설학기"].str.contains("-summer", case=False, na=False)]
    builtins.__dict__[f"df_{major_name}_winter"] = df_major[df_major["개설학기"].str.contains("-winter", case=False, na=False)]


## 4. 계절별 전공 개설학기 보기

### 4-1. 연계전공

#### 4-1-1. 빅데이터사이언스 연계전공

In [33]:
bds_codes = [
    "STS2011", "MAT2110", "MAT3020", "MGT2002", "ECO2004", 
    "BDS4010", "CSW4010", "CSE4187",
    "AIC4012", "BDS3010", "BDS3020", "CSE4130", "CSW2010", "CSW2020",
    "CSW2030", "CSE3080", "CSW2050", "CSW3010", "CSE3081", "CSW3030", "CSE4110",
    "CSW3060", "CSW3080",
    "CSW4020", "ECO2009", "ECO3022", "ECO3023", "ECO4003", "ECO4004",
    "ECO4032", "EEE4178", "JAS4014", "MAS1004", "MAS2009",
    "MAS2010", "MAT3110", "MAT4331", "MGT4202", "MGT4208", "MGT4226",
    "MGT4515", "MGT4517", "MGT6613", "MGTG613"
]

make_major_dfs(df_all, major_name="bds", codes=bds_codes)

In [39]:
#df_bds_spring
#df_bds_summer
df_bds_fall
#df_bds_winter

,과목번호,과목명,교수,개설학기
0,BDS3020,블록체인및응용,문현아,"2023-2, 2024-2, 2025-2"
1,BDS4010,빅데이터종합설계(캡스톤디자인),"최수진, 하정욱","2023-2, 2024-2, 2025-2"
2,CSE3080,자료구조,"소정민, 양지훈, 장부루, 장형수, 정성원, 정영민, 최준석","2023-1, 2023-2, 2024-1, 2024-2, 2024-summer, 2024-winter, 2025-1, 2025-2, 2025-summer"
3,CSE3081,알고리즘설계와분석,"김세준, 서양진, 소정민, 임인성, 장형수","2023-1, 2023-2, 2024-2, 2025-2, 2025-summer"
4,CSE4187,"산학프로젝트(캡스톤디자인), 캡스톤디자인II(캡스톤디자인)","구명완, 장두성, 정영민","2023-1, 2023-2, 2024-1, 2024-2, 2025-1, 2025-2"
5,CSW2020,기초Java언어,강호석,"2023-2, 2024-2, 2025-2"
6,CSW2030,자료구조입문,임종석,"2023-2, 2024-2, 2025-2"
7,CSW2050,코퍼스언어학,홍정하,"2023-2, 2024-2, 2025-2"
8,CSW3030,데이터베이스입문,박석,"2023-2, 2024-2, 2025-2"
9,ECO2004,경제통계학,"김인경, 박경옥, 백예인, 사공용, 이성원, 이영훈, 주하연","2023-1, 2023-2, 2023-winter, 2024-1, 2024-2, 2024-winter, 2025-1, 2025-2"


#### 4-1-2. 공공인재 연계전공

In [18]:
pub_codes = [
    "PUB2005", "POL3130", "POL2002", "SOC2001", "SOC2003", "SOC3010",
    "ECO2001", "ECO2002", "ECO3009", "ECO3011", "ECO3017", "MGT2002", 
    "MGT2003", "PHI2005", "PSY2001", "PSY3009", "PUB3030", "PUB3016",
    "PUB3029", "PUB3023", "PUB3024", "PUB3025", "PUB3031", "PUB3032",
    "PUB3026", "PUB3021", "PUB3020", "PUB3022", "PUB3028", "PUB3027",
    "PUB4009", "KOR4500", "EDU2001", "PHI4010", "ECO2007", "MGT3004",
    "MGT4301", "MGT3005", "MGT4404"
]

make_major_dfs(df_all, major_name="pub", codes=pub_codes)


In [19]:
#df_pub_spring
#df_pub_summer
df_pub_fall
#df_pub_winter

,과목번호,과목명,교수,개설학기
0,ECO2001,경제학원론I,"Iris Haydar Doruk, 김경환, 김숙영, 김영철, 김진원, 김홍균, 손성...","2023-1, 2023-2, 2023-summer, 2023-winter, 2024..."
1,ECO2002,경제학원론II,"곽노선, 김경환, 김숙영, 김재호, 김형욱, 송의영, 전현배, 정국모, 허준영","2023-1, 2023-2, 2023-summer, 2023-winter, 2024..."
2,ECO2007,거시경제학I,"곽노선, 곽준희, 김숙영, 남주하, 이윤수, 정국모, 조윤호, 허준영","2023-1, 2023-2, 2024-1, 2024-2, 2025-1, 2025-2"
3,ECO3011,국제금융론,"곽준희, 김윤정","2023-1, 2023-2, 2024-1, 2024-2, 2025-2"
4,ECO3017,국제경제학,"곽준희, 김윤정","2023-1, 2023-2, 2024-1, 2024-2, 2025-1"
5,EDU2001,교육학개론,"김재웅, 신효정, 이예경","2023-1, 2023-2, 2024-1, 2024-2, 2025-1, 2025-2"
6,MGT2002,경영통계학,"김명석, 김범수, 이군희, 이윤동, 정예숙, 조성빈","2023-1, 2023-2, 2023-summer, 2023-winter, 2024..."
7,MGT2003,회계학원론,"강평경, 김민, 김아람, 박두리, 박재형, 변상혁, 송민섭, 양준선, 이다혜, 천준범","2023-1, 2023-2, 2023-summer, 2023-winter, 2024..."
8,MGT3004,재무관리,"김도성, 박영석, 안성필, 오동석, 이상호, 이영주, 이정진, 홍광헌","2023-1, 2023-2, 2024-1, 2024-2, 2025-1, 2025-2..."
9,MGT3005,관리회계,"김민, 박예연, 박재형, 변상혁, 양준선, 황국재","2023-1, 2023-2, 2024-1, 2024-2, 2025-1, 2025-2"


#### 4-1-3. 교직

In [20]:
edu_codes = [
    "EDU2001", "EDU2002", "EDU2003", "EDU2004", "EDU3001", "EDU3047",
    "EDU3002", "EDU3033", "EDU3045", "EDU3046", "EDU3037", "EDU3004", "EDU3035",
    "EDU2005", "EDU2007", "EDU3038", "EDU3039", "EDU3048", "EDU3049",
    "SHU4019", "SHU4022", "SHU4031", "EDU3036"
]

make_major_dfs(df_all, major_name="edu", codes=edu_codes)

In [21]:
#df_edu_spring
#df_edu_summer
df_edu_fall
#df_edu_winter

,과목번호,과목명,교수,개설학기
0,EDU2001,교육학개론,"김재웅, 신효정, 이예경","2023-1, 2023-2, 2024-1, 2024-2, 2025-1, 2025-2"
1,EDU2002,교육철학및교육사,한석훈,"2023-1, 2023-2, 2024-1, 2024-2, 2025-1, 2025-2"
2,EDU2003,교육심리,"양미경, 이예경","2023-2, 2024-2, 2025-1"
3,EDU3033,교육사회,김영미,"2023-2, 2024-2, 2025-2"
4,EDU3035,교육봉사활동,"신효정, 양미경","2023-1, 2023-2, 2023-summer, 2023-winter, 2024..."
5,EDU3037,생활지도및상담,"계은경, 한기백","2023-1, 2023-2, 2024-1, 2024-2, 2025-1, 2025-2"
6,EDU3045,특수교육학개론,박미경,"2023-1, 2023-2, 2024-1, 2024-2, 2025-1, 2025-2"
7,EDU3046,교직실무,"김연후, 김재웅, 이미숙","2023-1, 2024-2, 2025-2"
8,EDU3048,빅데이터의이해와교육적활용(캡스톤디자인),신효정,"2023-2, 2024-2, 2025-2"
12,EDU3001,교육과정,양미경,"2023-1, 2024-1, 2025-2"


### 4-2. 일반 전공

#### 4-2-1. 경영

In [14]:
make_major_dfs(df_all, major_name="mgt", prefix="MGT")

In [15]:
#df_mgt_spring
#df_mgt_summer
df_mgt_fall
#df_mgt_winter

,과목번호,과목명,교수,개설학기
0,MGT2002,경영통계학,"김명석, 김범수, 이군희, 이윤동, 정예숙, 조성빈","2023-1, 2023-2, 2023-summer, 2023-winter, 2024..."
1,MGT2003,회계학원론,"강평경, 김민, 김아람, 박두리, 박재형, 변상혁, 송민섭, 양준선, 이다혜, 천준범","2023-1, 2023-2, 2023-summer, 2023-winter, 2024..."
2,MGT3001,조직행동이론,"양동훈, 이인석, 이해경, 장영균, 조봉순, 최장호","2023-1, 2023-2, 2024-1, 2024-2, 2025-1, 2025-2"
3,MGT3002,경영정보시스템,"김용진, 김태연, 오혜림, 이상근, 한재형","2023-1, 2023-2, 2024-1, 2024-2, 2025-1, 2025-2"
4,MGT3003,"생산관리론, 운영관리","강세원, 김길선, 김민균, 김수효, 하병천","2023-1, 2023-2, 2024-1, 2024-2, 2024-summer, 2..."
...,...,...,...,...
84,MGTG708,경영학 연구에서의 경제적 모델링,김길선,2025-2
85,MGT4245,정보화시대의기업경영과혁신,구자현,"2023-2, 2024-1"
86,MGTG109,기업지배구조와기업전략,서정일,"2023-2, 2024-1"
88,MGT4308,벤처캐피탈과투자분석(캡스톤디자인),안성필,2023-2


#### 4-2-2. 국문

In [16]:
make_major_dfs(df_all, major_name="kor", prefix="KOR")

In [17]:
#df_kor_spring
#df_kor_summer
df_kor_fall
#df_kor_winter

,과목번호,과목명,교수,개설학기
0,KOR2001,국어학입문,"이정훈, 이지영","2023-1, 2023-2, 2024-1, 2024-2, 2025-1, 2025-2"
1,KOR2200,국문학개설,"백진우, 이정원, 황윤정","2023-1, 2023-2, 2024-1, 2024-2, 2025-1, 2025-2"
2,KOR2201,문학이란무엇인가,"김경수, 박숙자, 박슬기, 우찬제","2023-1, 2023-2, 2024-1, 2024-2, 2025-1, 2025-2"
3,KOR3007,국어담화화용론,명정희,"2023-2, 2024-2, 2025-2"
4,KOR3015,국어사,이지영,"2023-1, 2024-2"
5,KOR3201,한문I,백진우,"2024-2, 2025-2"
6,KOR3204,국문학과전통문화,최지혜,"2023-2, 2024-2, 2025-2"
7,KOR3305,고전시가론,최지혜,"2023-2, 2024-2, 2025-2"
8,KOR3402,현대소설텍스트읽기I,김경수,"2023-1, 2024-2"
9,KOR3413,영상문학론,박미란,"2023-2, 2024-2"


#### 4-2-3. 경제

In [22]:
make_major_dfs(df_all, major_name="eco", prefix="ECO")

In [23]:
#df_eco_spring
#df_eco_summer
df_eco_fall
#df_eco_winter

,과목번호,과목명,교수,개설학기
0,ECO2001,경제학원론I,"Iris Haydar Doruk, 김경환, 김숙영, 김영철, 김진원, 김홍균, 손성...","2023-1, 2023-2, 2023-summer, 2023-winter, 2024..."
1,ECO2002,경제학원론II,"곽노선, 김경환, 김숙영, 김재호, 김형욱, 송의영, 전현배, 정국모, 허준영","2023-1, 2023-2, 2023-summer, 2023-winter, 2024..."
2,ECO2003,경제수리기초,"김인경, 김형욱, 박경옥, 이강오","2023-1, 2023-2, 2023-summer, 2024-1, 2024-2, 2..."
3,ECO2004,경제통계학,"김인경, 박경옥, 백예인, 사공용, 이성원, 이영훈, 주하연","2023-1, 2023-2, 2023-winter, 2024-1, 2024-2, 2..."
4,ECO2006,미시경제학I,"Iris Haydar Doruk, 김도영, 김인경, 왕규호, 이강오, 이진권, 주하연","2023-1, 2023-2, 2024-1, 2024-2, 2025-1, 2025-2"
5,ECO2007,거시경제학I,"곽노선, 곽준희, 김숙영, 남주하, 이윤수, 정국모, 조윤호, 허준영","2023-1, 2023-2, 2024-1, 2024-2, 2025-1, 2025-2"
6,ECO2008,수리경제학,김영철,"2023-2, 2023-winter, 2024-2, 2025-2"
7,ECO2009,계량경제학I,"김재호, 남준우, 백예인, 이성원, 이영훈, 이한식","2023-1, 2023-2, 2024-1, 2024-2, 2025-1, 2025-2"
8,ECO3001,산업경제학,"전현배, 주하연","2023-1, 2023-2, 2024-1, 2024-2, 2025-1, 2025-2"
9,ECO3004,도시경제학,김진원,"2023-2, 2024-2, 2025-2"


#### 4-2-4. 수학

In [24]:
make_major_dfs(df_all, major_name="mat", prefix="MAT")

In [25]:
#df_mat_spring
#df_mat_summer
df_mat_fall
#df_mat_winter

,과목번호,과목명,교수,개설학기
0,MAT1060,미적분학실습II,"김현석, 임경수","2023-2, 2024-2, 2025-2"
1,MAT2010,집합론,"김순영, 김현석, 박준용, 신창언, 이종범, 하구용","2023-1, 2023-2, 2023-summer, 2023-winter, 2024..."
2,MAT2110,선형대수학,"강유태, 김대욱, 김준태, 박준용, 안소영, 옥지훈, 이영란, 조상현, 조성희, 조...","2023-1, 2023-2, 2023-summer, 2023-winter, 2024..."
3,MAT2120,정수론,"김순영, 김지현, 김현정, 박준용, 조창연, 최승일","2023-2, 2023-summer, 2023-winter, 2024-2, 2024..."
4,MAT2220,고등미적분학II,"김현석, 옥지훈, 이재성","2023-2, 2024-2, 2025-2"
5,MAT2230,미분방정식,"김현석, 신창언, 옥지훈, 이영록, 조상현, 한상현","2023-2, 2023-summer, 2023-winter, 2024-2, 2024..."
6,MAT2420,응용수학II,"안소영, 유지현","2023-2, 2023-summer, 2024-2, 2024-summer, 2025..."
7,MAT3020,통계학입문,"임경수, 임경필","2023-1, 2023-2, 2023-summer, 2023-winter, 2024..."
8,MAT3310,다양체기하학,"김종수, 김준태","2023-2, 2024-2, 2025-2"
9,MAT3410,확률론입문,"이재성, 임경필","2023-2, 2024-2, 2025-2"
